In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
#pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
donors = pd.read_csv('donors.csv')
donors.drop('Unnamed: 0', axis = 1, inplace  = True)
donors.head()

D:\Programas\Anaconda\envs\dmi\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (9,412) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,2009-01-01,GRI,0,IL,61081,,,1957-12-01,0,,...,7.741935,95515,0,L,4,E,X,X,X,C
1,2014-01-01,BOA,1,CA,91326,,,1972-02-01,0,,...,15.666667,148535,0,L,2,G,X,X,X,A
2,2010-01-01,AMH,1,NC,27017,,,NaN,0,,...,7.481481,15078,1,L,4,E,X,X,X,C
3,2007-01-01,BRY,0,CA,95953,,,1948-01-01,0,,...,6.812500,172556,1,L,4,E,X,X,X,C
4,2006-01-01,,0,FL,33176,,,1940-01-01,0,X,...,6.864865,7112,1,L,2,F,X,X,X,A


# start


In [5]:
#BELO - COHERENCE
#it has more than 50 states bc it includes US territories like the Virgin Islands, Northern Mariana Islands, Guam, etc
donors['STATE'].nunique()

57

In [5]:
#no donors after today's date
donors['ODATEDW'] = pd.to_datetime(donors['ODATEDW'])
(donors['ODATEDW'] > datetime.datetime.now()).sum()

0

In [6]:
donors['NOEXCH'].value_counts()
#drop this! (incoherent, bc the variable is not very relevant and it 
#is not consistent with the metadata's variables explanation -> explicar

#melhor no relatório)

0    61203
0    33882
1      195
1       90
X       35
         7
Name: NOEXCH, dtype: int64

In [8]:
donors['GENDER'].value_counts() 
# A and C are incoherent, we will assume as U: Unknown

F    51277
M    39094
      2957
U     1715
J      365
C        2
A        2
Name: GENDER, dtype: int64

In [21]:
print('< 0: \n', (donors.iloc[:, 40:47] < 0).sum())
print('> 100: \n', (donors.iloc[:, 40:47] > 100).sum())

#no incoherence

< 0: 
 MALEMILI    0
MALEVET     0
VIETVETS    0
WWIIVETS    0
LOCALGOV    0
STATEGOV    0
FEDGOV      0
dtype: int64
> 100: 
 MALEMILI    0
MALEVET     0
VIETVETS    0
WWIIVETS    0
LOCALGOV    0
STATEGOV    0
FEDGOV      0
dtype: int64


In [4]:
donors = donors[donors['AGE902'] != 0]

In [50]:
donors['AGE906'].min()

27

In [ ]:
#PROBLEM on AGE902: "Median Age of Adults 18 or Older" but it had a value that was 0... actually, after analyzing the 
#subset of donors that had AGE902 equal to 0, we found out that the vast majority of them had also 0 in basically all
#socio-demographic variables. This leads us to belive taht the missing values on this columns may be represented by a 0 other than
#a Numpy NaN. Anyways, these records have incoherences, and for this reason, they should be removed (799 records!).

#They have donations... but they should be removed anyways. 
#bc it would be a lot of records and diferente variables to predict

#after removing this records other incoherences in other variables were also removed

In [61]:
(donors.iloc[:, 75:358]).min()
#(donors.iloc[:, 75:358]).max()

## percentagens sem incoerencias (tudo entre 0-10), nao sei se precisamos incluir


#  Minimos:
#HHP1/2 == 0 doenst make sense
#HV1/2 a house with 0 value ??
#4/5 rent == 0 ?? 
#RPH's dont make sense to have avg of 0 rooms per housing unit 
#IC1:5 also doesnt make sense we have avg and medians of value 0
#TPE10/11 MEDIAN == 0 ??
#EC1 lowest values == 89 ?? doesnt make any sense 
#MHUC1/2 also minimum value of 0

#most of median minimum values == 0 doesnt make sense.. DROP?????


# Maximos:
# HHP1/2 == 650/700 how is that possible??
# RPH's value are too high 85/90/61/40
## EC1 -- 170 years of median, impossible
#MHUC1/2 max value 21/5 values are too low 


#I think most or all median values are incorrect. 
#Some are impossible and other are possible but dont make sense I a real world scenario
#for that I think they should be removed and use only the percentages for our analysis

POP90C1        99.0
POP90C2        99.0
POP90C3        99.0
POP90C4        99.0
POP90C5        99.0
ETH1           99.0
ETH2           99.0
ETH3           99.0
ETH4           99.0
ETH5           99.0
ETH6           22.0
ETH7           72.0
ETH8           99.0
ETH9           67.0
ETH10          46.0
ETH11          47.0
ETH12          72.0
ETH13          97.0
ETH14          57.0
ETH15          81.0
ETH16          86.0
AGE901         84.0
AGE902         84.0
AGE903         84.0
AGE904         84.0
AGE905         84.0
AGE906         84.0
AGE907         75.0
CHIL1          99.0
CHIL2          99.0
CHIL3          99.0
AGEC1          99.0
AGEC2          99.0
AGEC3          99.0
AGEC4          99.0
AGEC5          99.0
AGEC6          99.0
AGEC7          99.0
CHILC1         99.0
CHILC2         99.0
CHILC3         99.0
CHILC4         99.0
CHILC5         99.0
HHAGE1         99.0
HHAGE2         99.0
HHAGE3         99.0
HHN1           99.0
HHN2           99.0
HHN3           99.0
HHN4           99.0


In [ ]:
#PROBLEM on HV1: "Median Home Value in hundreds" has values of 0, and then, the second lowest, 75 (which means 7500)... 
#the values of 0 are incoherences!!? (Belo) (318 records - same on HV2)


In [ ]:
#EC1: "Median Years of School Completed by Adults 25+" has minimum 89 and maximum 170... are the values wrong?

#HC1: "Percent Median Length of Residence" is percent and median at the same time? And has values of 0 length! Is it the 
#median time someone has been living in their current home, in years? Very ambiguous:
#HC2: this one has a less ambiguous description but it still says "Percent Median" so is it a percentage or a median? 
#it is confusing

In [67]:
donors['ODATEDW'].head()

0    2009-01-01
1    2014-01-01
2    2010-01-01
3    2007-01-01
4    2006-01-01
Name: ODATEDW, dtype: object

In [11]:
#now the dates are datetime
donors['FISTDATE'] = pd.to_datetime(donors['FISTDATE'], errors='ignore')

#            FISTDATE:  Date of first gift 
#                cant be before 
#   ODATEDW: Origin Date. Date of donor's first gift to PVA

#                  RIGHT ?!!!!!!!!!!!!!????????????

(donors['FISTDATE'] < donors['ODATEDW']).sum()

17445